In [1]:
import socket 
import threading
import cv2
import numpy as np

sock= socket.socket()
sock.bind(("",8081))

sender_ip=input("Enter the Sender Ip : ")
sender_port=int(input("Enter the Sender port : "))
sock.connect((sender_ip,sender_port))
cameraIndex = 0
camera = cv2.VideoCapture(cameraIndex)


def receiver():
    while True:
        data = sock.recv(10000000000)
        if(data == b'finished'):
            print("Finished")
            camera.release()
            sock.close()
        else:  
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3:
                cv2.imshow('From Server', photo.reshape(480, 640, 3))
            if cv2.waitKey(80) == 13: 
                sock.send(b'finished')
                camera.release()
                cv2.destroyAllWindows()
                exit()

def sender():
    while True: 
        status, photo = camera.read()
        photo = cv2.resize(photo, (640, 480))
        if status:
            sock.send(np.ndarray.tobytes(photo))
        else:
            print("Not in frame")
    camera.release()


threading.Thread(target=receiver).start()
threading.Thread(target=sender).start()

Enter the Sender Ip : 192.168.56.1
Enter the Sender port : 1234


In [ ]:
camera.release()